In [68]:
import numpy as np
import nltk
nltk.download('punkt')
import pandas as pd
import re

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [120]:
# Read in synthetic data
df = pd.read_csv('../../data/data/generated/data_230124-172021.csv')

In [121]:
df['text_split'] = df['text'].apply(lambda x: x.strip())
df['text_split'] = df['text_split'].apply(lambda x: nltk.word_tokenize(x))

string_cols = ["item1", "item2", ]
df[string_cols] = df[string_cols].astype(str)
df.head()
# df.info()

,location,organization,type,date,unit,weight1,item1,prompt,text,weight2,item2,text_split
0,Adams Rocks,Take 3,instagram caption,2017-04-07,kilograms,200,spread tubs,Generate an instagram caption for a beach clea...,It was inspiring to witness so many people com...,NaN,nan,"[It, was, inspiring, to, witness, so, many, pe..."
1,Grahams Beach,Global Alliance Against Marine Pollution,instagram caption,2018-09-04,pounds,100,trash,Generate an instagram caption for a beach clea...,We just made a huge difference at Grahams Beac...,NaN,nan,"[We, just, made, a, huge, difference, at, Grah..."
2,Norfolk Island,Plastic Pollution Coalition Australia,press release,2021-05-25,kilograms,386,glass bottles,Generate a press release for a beach cleanup w...,\nPlastic Pollution Coalition Australia (PPCA)...,NaN,nan,"[Plastic, Pollution, Coalition, Australia, (, ..."
3,Playa Grande de Saboga,Take 3,press release,2016-12-30,kgs,332,tupperwares,Generate a press release for a beach cleanup w...,Take 3 Celebrates a Successful Beach Cleanup i...,NaN,nan,"[Take, 3, Celebrates, a, Successful, Beach, Cl..."
4,Playa de Chachalacas,Rameau Project,press release,2022-04-15,lbs,168,plastic,Generate a press release for a beach cleanup w...,FOR IMMEDIATE RELEASE\n\nThe Rameau Project Ce...,NaN,nan,"[FOR, IMMEDIATE, RELEASE, The, Rameau, Project..."


In [124]:
# Convert data into list of words with associated 'B - entity', 'I - entity' or 'O'
# Look at other preprocessing steps in read_datasets function in convert_bn_daffodil

units = ["kilograms", "kilogram", "kgs", "kg", "lb", "lbs", "pounds", "pound"]

def assign_entity_types(row):
    sentence = row['text_split']
    new_tags = []
    prev_org_tag = False
    prev_loc_tag = False
    prev_unit_tag = False
    prev_weight_tag = False
    prev_item_tag = False
    for i in range(len(sentence)):
        # Assign location labels
        if any(sentence[i] == word for word in row['location'].split()): #Problem: only searches for specific location that was 
                                        # given in the prompt, not other locations that GPT-3 produces in text
                                        # Solution: implement another check for all international countries?
            if prev_loc_tag:
                new_tags.append("I-LOC")
            else:
                new_tags.append("B-LOC")
                prev_loc_tag = True
        # Assign organization labels
        elif any(sentence[i] == word for word in row['organization'].split()):
            if prev_org_tag:                    # consider capitalization - should i lowercase everything when checking orgs?
                new_tags.append("I-ORG")
            else:
                new_tags.append("B-ORG")
                prev_org_tag = True
        # Assign unit labels
        elif any(sentence[i] == word for word in units):
            if prev_unit_tag: 
                new_tags.append("I-UNT")
            else:
                new_tags.append("B-UNT")
                prev_unit_tag = True
        # Assign weight labels 
        elif (sentence[i] == str(row['weight1']) or (row['weight2'] != None and sentence[i] == str(row['weight2']))):
            if prev_weight_tag: 
                new_tags.append("I-WEI")
            else:
                new_tags.append("B-WEI")
                prev_weight_tag = True
        # Assign item labels
        elif (any(sentence[i] == word for word in row['item1'].split()) or 
                                (row['item2'] != None and any(sentence[i] == word for word in row['item2'].split()))):
            if prev_item_tag: 
                new_tags.append("I-ITM")
            else:
                new_tags.append("B-ITM")
                prev_item_tag = True
        # Open question: How to assign dates? Need to capture all possible date formats?
        else:
            new_tags.append("O")
            prev_org_tag = False
            prev_loc_tag = False
            prev_unit_tag = False
            prev_weight_tag = False
            prev_item_tag = False
    return list(zip(sentence, new_tags))

df['tagged_entities'] = df.apply(lambda row : assign_entity_types(row), axis =1)


In [125]:
# Review newly assigned non-"O" tags
for i in df.iloc[2]['tagged_entities']:
    if i[1] != "O":
        print(i[0], i[1])

print(df['tagged_entities'][2])
# df['text'][2]

Plastic B-ORG
Pollution I-ORG
Coalition I-ORG
Australia I-ORG
Norfolk B-LOC
Island I-LOC
386 B-WEI
kilograms B-UNT
glass B-ITM
bottles I-ITM
Australia B-ORG
glass B-ITM
bottles I-ITM
Australia B-ORG
[('Plastic', 'B-ORG'), ('Pollution', 'I-ORG'), ('Coalition', 'I-ORG'), ('Australia', 'I-ORG'), ('(', 'O'), ('PPCA', 'O'), (')', 'O'), ('is', 'O'), ('proud', 'O'), ('to', 'O'), ('announce', 'O'), ('the', 'O'), ('success', 'O'), ('of', 'O'), ('its', 'O'), ('latest', 'O'), ('beach', 'O'), ('cleanup', 'O'), ('on', 'O'), ('Norfolk', 'B-LOC'), ('Island', 'I-LOC'), ('.', 'O'), ('On', 'O'), ('May', 'O'), ('25th', 'O'), (',', 'O'), ('2021', 'O'), (',', 'O'), ('PPCA', 'O'), ('and', 'O'), ('its', 'O'), ('volunteers', 'O'), ('collected', 'O'), ('a', 'O'), ('whopping', 'O'), ('386', 'B-WEI'), ('kilograms', 'B-UNT'), ('of', 'O'), ('glass', 'B-ITM'), ('bottles', 'I-ITM'), ('from', 'O'), ('the', 'O'), ('island', 'O'), ("'s", 'O'), ('coastline', 'O'), ('.', 'O'), ('This', 'O'), ('is', 'O'), ('a', 'O'), ('re

In [2]:
# Divide data into datasets = (train_sentences, dev_sentences, test_sentences)

In [4]:
# Convert to JSON file needed by Stanza model
# There is a conversion script called several times in prepare_ner_dataset.py which converts IOB format to our internal NER format:
# import stanza.utils.datasets.ner.prepare_ner_file as prepare_ner_file

# prepare_ner_file.process_dataset(input_iob, output_json)

In [ ]:
# Or call write_dataset function from stanza.utils.datasets.ner.utils (docstring below)?
def write_dataset(datasets, output_dir, short_name, suffix="bio"):
    """
    write all three pieces of a dataset to output_dir

    datasets should be 3 lists: train, dev, test
    each list should be a list of sentences
    each sentence is a list of pairs: word, tag

    after writing to .bio files, the files will be converted to .json
    """
    pass